# Selenium Example Notebook


### Step 1: Set up Selium Driver

In [15]:
#Load selinimum and automatically install the Chrome Driver
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time

install = ChromeDriverManager().install()
install

'C:\\Users\\24731\\.wdm\\drivers\\chromedriver\\win32\\114.0.5735.90\\chromedriver.exe'

In [16]:
import pandas as pd
import re

In [17]:
#Setup Chrome driver 
options = Options()
#options.add_argument('--headless')
driver = webdriver.Chrome(options=options)
time.sleep(10)

### Step 2: Call the main Course website for MSU 

In [18]:
from bs4 import BeautifulSoup

url = "https://student.msu.edu/psc/public/EMPLOYEE/SA/c/NUI_FRAMEWORK.PT_AGSTARTPAGE_NUI.GBL?CONTEXTIDPARAMS=TEMPLATE_ID%3aPTPPNAVCOL&scname=MSU_AA_SCHEDULE_NEW0&PanelCollapsible=Y"
driver.get(url)
time.sleep(20)

<a class="ps-link" href="" id="SSR_CSTRMPRV_VW_DESCR$3" onclick="javascript:cancelBubble(event);" ptlinktgt="pt_peoplecode" tabindex="-1">

###Step 3: Pick a semester

In this example you can try the previous, current or next

In [19]:
Semester = {'Summer 2020': "'SSR_CSTRMPRV_VW_DESCR$8'",
            'Fall 2020': "'SSR_CSTRMPRV_VW_DESCR$span$7'",
            'Spring 21': "'SSR_CSTRMPRV_VW_DESCR$span$6'",
            'Summer 21': "'SSR_CSTRMPRV_VW_DESCR$5'",
            'Fall 21': "'SSR_CSTRMPRV_VW_DESCR$4'",
            'Spring 22': "'SSR_CSTRMPRV_VW_DESCR$span$3'",
            'Summer 22': "'SSR_CSTRMPRV_VW_DESCR$2'",
            'Fall 22': "'SSR_CSTRMPRV_VW_DESCR$1'",
            'Spring 23': "'SSR_CSTRMPRV_VW_DESCR$0'",
            'Summer 23': "'SSR_CSTRMCUR_VW_DESCR$0'", 
            'Fall 23': "'SSR_CSTRMCUR_VW_DESCR$1'",
           'Spring 24': "'SSR_CSTRMCUR_VW_DESCR$2'",
           'Summer 24': "'SSR_CSTRMCUR_VW_DESCR$3'"}

            
url = f"javascript:submitAction_win0(document.win0,{Semester['Spring 23']});"

In [20]:
#url = "javascript:submitAction_win0(document.win0,'SSR_CSTRMPRV_VW_DESCR$3');"
driver.execute_script(url);
time.sleep(30)

In [21]:
"""body = driver.page_source
soup = BeautifulSoup(body, 'html.parser')
soup"""

"body = driver.page_source\nsoup = BeautifulSoup(body, 'html.parser')\nsoup"

#### Step 4: Type in a form value to the Subject element (I had to find the name) and then hit submit!

In [22]:
from selenium.webdriver.common.by import By
element = driver.find_element(By.ID, 'MSU_CLSRCH_WRK2_SUBJECT')  
element.send_keys("CMSE")
# element.get_property('value')

In [23]:
url = f"javascript:submitAction_win0(document.win0,'MSU_CLSRCH_WRK_SSR_PB_SEARCH');"
driver.execute_script(url);
time.sleep(20)

In [11]:
#body = driver.page_source
#soup = BeautifulSoup(body, 'html.parser')
#soup

In [11]:
""""element = driver.find_element(By.XPATH, "//div[contains(@class, 'ps_box-htmlarea psc_label-suppressed psc_label-none psc_label-halignleft')]")

# Click on the element
element.click()
time.sleep(30""""

In [24]:
def get_class_info(soup):
    values = []
    divs = soup.find_all("div", class_="ps-htmlarea")
    for div in divs:
        val = (div.get_text(strip=True))
        values.append(val)
    values = list(filter(lambda x: x != "", values))
    values.pop(0)
    reshaped_list = [values[i:i+6] for i in range(0, len(values), 6)]

    # Create a DataFrame from the reshaped list
    col_names = ['Course', 'Type', 'Section', 'Schedule', 'Dates', 'Instructor']
    df = pd.DataFrame(reshaped_list, columns=col_names)
    df[['Course Code', 'Course Name']] = df['Course'].str.split(':', 1, expand=True)
    df[['Type', 'Units']] = df['Type'].str.split('(', 1, expand=True)
    df[['Section', 'Class Nbr', 'Academic Session']] = df['Section'].str.split('/', 2, expand=True)
    df[['Days', 'Time']] = df['Schedule'].str.split(':', 1, expand=True)
    df[['Units','Status']] = df['Units'].str.split(')',1,expand=True)
    df[['Subject','Course Number']] = df['Course Code'].str.split(' ',1,expand=True)

    df = df.drop(['Course', 'Schedule','Course Code'], axis=1)
    df = df[['Subject','Course Number','Course Name','Type','Units','Status','Section','Class Nbr','Academic Session','Days','Time','Dates','Instructor']]
    df['Units'] = df['Units'].str.extract(r'(\d+(?:\.\d+)?)')
    df['Section'] = df['Section'].str.extract(r'(\d+(?:\.\d+)?)')
    df['Class Nbr'] = df['Class Nbr'].str.extract(r'(\d+(?:\.\d+)?)')
    return df

In [29]:
element = driver.find_element(By.ID, "DESCR100$0_row_1") #testing on a single class
element.click()
time.sleep(20)
#cancel_cmd="javascript:doUpdateParent(document.win0,'#ICCancel');"
#driver.execute_script(cancel_cmd);
#body = driver.page_source
soup = BeautifulSoup(body, 'html.parser')
df = get_class_info(soup)
df

,Subject,Course Number,Course Name,Type,Units,Status,Section,Class Nbr,Academic Session,Days,Time,Dates,Instructor
0,CMSE,180,Introduction to Data ScienceCross-Listed,Lecture,4.00,Open,001,36448,Regular Academic Session,Mon Wed,10:20 AM-12:10 PM,01/09/2023 - 04/30/2023,D Caballero
1,CMSE,201,Computational Modeling and Data Analysis I,Lecture,4.00,Wait List,002,11842,Regular Academic Session,Mon Wed,10:20 AM-12:10 PM,01/09/2023 - 04/30/2023,D Silvia
2,CMSE,201,Computational Modeling and Data Analysis I,Lecture,4.00,Open,003,11843,Regular Academic Session,Mon Wed,12:40 PM-2:30 PM,01/09/2023 - 04/30/2023,H Lei
3,CMSE,201,Computational Modeling and Data Analysis I,Lecture,4.00,Open,004,11844,Regular Academic Session,Mon Wed,3:00 PM-4:50 PM,01/09/2023 - 04/30/2023,P Hamerski
4,CMSE,201,Computational Modeling and Data Analysis I,Lecture,4.00,Open,005,11845,Regular Academic Session,Tue Thu,8:00 AM-9:50 AM,01/09/2023 - 04/30/2023,L Silvestri
5,CMSE,201,Computational Modeling and Data Analysis I,Lecture,4.00,Wait List,006,11846,Regular Academic Session,Tue Thu,10:20 AM-12:10 PM,01/09/2023 - 04/30/2023,S Ravishankar
6,CMSE,201,Computational Modeling and Data Analysis I,Lecture,4.00,Open,007,11847,Regular Academic Session,Tue Thu,12:40 PM-2:30 PM,01/09/2023 - 04/30/2023,Y Cheng
7,CMSE,201,Computational Modeling and Data Analysis I,Lecture,4.00,Wait List,008,11848,Regular Academic Session,Tue Thu,3:00 PM-4:50 PM,01/09/2023 - 04/30/2023,L Silvestri
8,CMSE,202,Computational Modeling and Data Analysis II,Lecture,4.00,Wait List,001,11850,Regular Academic Session,Mon Wed,10:20 AM-12:10 PMMon Wed : 10:20 AM-12:10 PM,01/09/2023 - 04/30/202301/09/2023 - 04/30/2023,H Yu
9,CMSE,202,Computational Modeling and Data Analysis II,Lecture,4.00,Wait List,001,11850,Regular Academic Session,Mon Wed,10:20 AM-12:10 PMMon Wed : 10:20 AM-12:10 PM,01/09/2023 - 04/30/202301/09/2023 - 04/30/2023,H Yu


# Other Userful Commands

In [30]:
#departments = element.text.split('\n')